In [1]:
import sys
sys.path.append('/root/autodl-tmp/CommitFit')
import pandas as pd
from sklearn.model_selection import train_test_split
import ensemble_model.preprocesser as preprocesser 
import ensemble_model.combined_model as cm 
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer, RobertaModel, RobertaTokenizer
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_curve

2026-01-06 16:31:00.946876: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-06 16:31:01.007709: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-06 16:31:02.235651: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/root/miniconda3/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8:

In [2]:
# =========================
# 0) 配置
# =========================
CSV_PATH = "/root/autodl-tmp/CommitFit/dataset/Ghadhab/dataset.csv"

# =========================
# 2) 读 CSV -> Dataset -> split
# =========================
df = pd.read_csv(CSV_PATH)
label2id={'Adaptive':0, 'Corrective':1, 'Perfective':2}
df = df.replace({"labels": label2id})
df

import re

def compress_diff_minimal(
    diff_text: str,
    max_changed_lines: int = 120,   # 最多保留多少条 +/-
    max_chars: int = 3500,          # 最终硬截断（字符）
) -> str:
    if not diff_text:
        return ""

    lines = diff_text.splitlines()
    kept = []
    changed_cnt = 0

    for ln in lines:
        # 文件头
        if ln.startswith("diff --git "):
            kept.append(ln)
            continue

        # 跳过噪声行
        if ln.startswith(("index ", "new file mode", "deleted file mode", "similarity index", "rename from", "rename to")):
            continue
        if ln.startswith(("--- ", "+++ ")):  # 这两行通常很长且重复文件名，可选保留；这里跳过以更“最小”
            continue
        if "GIT binary patch" in ln:
            continue

        # hunk 头
        if ln.startswith("@@"):
            kept.append(ln)
            continue

        # 只保留变更行（排除+++ / ---）
        if (ln.startswith("+") and not ln.startswith("+++")) or (ln.startswith("-") and not ln.startswith("---")):
            kept.append(ln)
            changed_cnt += 1
            if changed_cnt >= max_changed_lines:
                kept.append("... (diff truncated: too many changed lines)")
                break
            continue

        # 其他上下文行：不保留（最小改动版）
        # 如果你想保留少量上下文，把这里改成“遇到变更行附近保留1-2行”即可

    out = "\n".join(kept).strip()
    if len(out) > max_chars:
        out = out[:max_chars] + "\n... (diff truncated: max_chars)"
    return out


def build_prompt(diff_compact: str) -> str:
    return (
        "Write a concise Git commit message (imperative mood). "
        "Focus on what changed.\n\n"
        "Diff:\n"
        f"{diff_compact}\n\n"
        "Commit message:"
    )


df["diff_compact"] = df["diffs"].map(compress_diff_minimal)
df["gen_prompt"] = df["diff_compact"].map(build_prompt)   # 不要 label 版本

# df = df.rename(columns={'msgs':'target_text','gen_prompt':'source_text'})
label2id={'Adaptive':0, 'Corrective':1, 'Perfective':2}
df = df.replace({"labels": label2id})
df

,user,repo,commit,labels,msgs,diffs,feature,diff_compact,gen_prompt
0,ponsonio,RxJava,0531b8bff5c14d9504beefb4ad47f473e3a22932,2,Change hasException to hasThrowable--,diff --git a/rxjava-core/src/main/java/rx/Noti...,"[1, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/rxjava-core/src/main/java/rx/Noti...,Write a concise Git commit message (imperative...
1,ponsonio,RxJava,0950c46beda335819928585f1262dfe1dca78a0b,0,Trying to extend the Scheduler interface accor...,diff --git a/rxjava-core/src/main/java/rx/Sche...,"[2, 44, 0, 0, 30, 0, 0, 1, 18, 0, 0, 0, 0, 0, ...",diff --git a/rxjava-core/src/main/java/rx/Sche...,Write a concise Git commit message (imperative...
2,ponsonio,RxJava,0f92fdd8e6422d5b79c610a7fd8409d222315a49,0,RunAsync method for outputting multiple values--,diff --git a/rxjava-contrib/rxjava-async-util/...,"[2, 53, 0, 0, 42, 0, 0, 1, 45, 1, 0, 0, 0, 0, ...",diff --git a/rxjava-contrib/rxjava-async-util/...,Write a concise Git commit message (imperative...
3,ponsonio,RxJava,100f571c9a2835d5a30a55374b9be74c147e031f,1,forEach with Action1 but not Observer--I re-re...,diff --git a/language-adaptors/rxjava-groovy/s...,"[1, 5, 122, 9, 10, 9, 4, 1, 5, 18, 2, 0, 0, 0,...",diff --git a/language-adaptors/rxjava-groovy/s...,Write a concise Git commit message (imperative...
4,ponsonio,RxJava,191f023cf5253ea90647bc091dcaf55ccdce81cc,1,1.x: Fix Completable swallows- OnErrorNotImple...,diff --git a/src/main/java/rx/Completable.java...,"[1, 1, 0, 0, 0, 0, 0, 1, 21, 0, 0, 0, 0, 0, 0,...",diff --git a/src/main/java/rx/Completable.java...,Write a concise Git commit message (imperative...
...,...,...,...,...,...,...,...,...,...
1776,jenkinsci,clearcase-plugin,51e9da224f80254476a7dc446bca817b505381d8,2,Use a temporary file to decrease memory consum...,diff --git a/src/main/java/hudson/plugins/clea...,"[2, 12, 0, 4, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/src/main/java/hudson/plugins/clea...,Write a concise Git commit message (imperative...
1777,jexp,batch-import,609d6c4b1eea2c33d9fb950fcbb9ba9dc1f80fc3,2,added a more memory efficient structure for st...,diff --git a/src/main/java/org/neo4j/batchimpo...,"[10, 159, 29, 35, 9, 2, 1, 5, 106, 0, 4, 8, 0,...",diff --git a/src/main/java/org/neo4j/batchimpo...,Write a concise Git commit message (imperative...
1778,hdiv,hdiv,19b650c78a1c76f4fd90274d7f163f863c0d39e4,2,Memory and performance optimizations,diff --git a/hdiv-config/src/main/java/org/hdi...,"[31, 302, 131, 140, 170, 89, 53, 7, 88, 14, 17...",diff --git a/hdiv-config/src/main/java/org/hdi...,Write a concise Git commit message (imperative...
1779,casidiablo,persistence,d7bf95159df37a3d338ca267dddd3d26b38ec37c,2,Now it is possible to specify the sqlite open ...,diff --git a/pom.xml b/pom.xml\nindex 394263b....,"[5, 57, 20, 9, 21, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","diff --git a/pom.xml b/pom.xml\n@@ -23,5 +23,5...",Write a concise Git commit message (imperative...


In [3]:
train, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val, test = train_test_split(temp_df, test_size=0.5, random_state=42)

In [4]:
# train.to_csv('/root/autodl-tmp/COLARE-main/dataset/multi-lang-annotated/train.csv',index=False)
# test.to_csv('/root/autodl-tmp/COLARE-main/dataset/multi-lang-annotated/test.csv',index=False)
# val.to_csv('/root/autodl-tmp/COLARE-main/dataset/multi-lang-annotated/eval.csv',index=False)

In [5]:
# train = train.groupby("label", group_keys=False).apply(lambda x: x.sample(n=10, random_state=42))

In [6]:
# Load BERT and CodeBERT models and tokenizers
bert_model = BertModel.from_pretrained('/root/autodl-tmp/models/google-bert/bert-base-uncased')
bert_tokenizer = BertTokenizer.from_pretrained('/root/autodl-tmp/models/google-bert/bert-base-uncased')

codebert_model = RobertaModel.from_pretrained('/root/autodl-tmp/models/codebert-base')
codebert_tokenizer = RobertaTokenizer.from_pretrained('/root/autodl-tmp/models/codebert-base')

In [7]:
train

,user,repo,commit,labels,msgs,diffs,feature,diff_compact,gen_prompt
1507,GNOME,vala,ba7cd6426301af5e66f8537a4f1f1693bf192da1,0,glib-2.0: add GLib.IOChannel.win32_socket and ...,diff --git a/vapi/glib-2.0.vapi b/vapi/glib-2....,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/vapi/glib-2.0.vapi b/vapi/glib-2....,Write a concise Git commit message (imperative...
84,apache,hbase,4405698ee99fe26d0ac9317a2df96096f2731a7b,1,HBASE-7703 Eventually all online snapshots fai...,diff --git a/hbase-server/src/main/java/org/ap...,"[1, 1, 0, 0, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/hbase-server/src/main/java/org/ap...,Write a concise Git commit message (imperative...
1497,GNOME,vala,22b7e1b8fb3c0c7fc78decad7e918127ef32b43c,1,x11: fix type mismatch warning in X.Display.ge...,diff --git a/vapi/x11.vapi b/vapi/x11.vapi\nin...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/vapi/x11.vapi b/vapi/x11.vapi\n@@...,Write a concise Git commit message (imperative...
1001,GNOME,valadoc,81151dab07508e1657ddf252a11672c0b1868e7a,0,Improve {@inheritDoc}\n,diff --git a/src/libvaladoc/content/blockconte...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/src/libvaladoc/content/blockconte...,Write a concise Git commit message (imperative...
1573,aeshell,aesh,2b1f4e70ecabd92348d797dc7a315978a9d40805,0,[AESH-53]\nadd support to find option name com...,diff --git a/src/main/java/org/jboss/aesh/cl/C...,"[5, 85, 26, 4, 26, 11, 1, 1, 9, 0, 6, 0, 0, 0,...",diff --git a/src/main/java/org/jboss/aesh/cl/C...,Write a concise Git commit message (imperative...
...,...,...,...,...,...,...,...,...,...
1130,tapiji,tapiji,fc21d266c205497596bbc4aecb0a2b62558d0e0d,2,updated licenses\n,diff --git a/at.ac.tuwien.inso.eclipse.i18n/Ta...,"[1, 278, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",diff --git a/at.ac.tuwien.inso.eclipse.i18n/Ta...,Write a concise Git commit message (imperative...
1294,apache,deltaspike,b0caefee80a42d6737b3a255316487b668fab104,0,DELTASPIKE-278 add category to Message API\n,diff --git a/deltaspike/core/api/src/main/java...,"[2, 12, 0, 1, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",diff --git a/deltaspike/core/api/src/main/java...,Write a concise Git commit message (imperative...
860,apache,camel,610115b44e10e5046b62a7dfad06dde18e0f83e7,2,added explicit generics to avoid possible- com...,diff --git a/camel-core/src/main/java/org/apac...,"[12, 0, 0, 19, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",diff --git a/camel-core/src/main/java/org/apac...,Write a concise Git commit message (imperative...
1459,GNOME,vala,b48b41f97dfc0c37251beb07b1924af593bba8e1,1,gsl: Do not use `weak' modifier where it is no...,diff --git a/vapi/gsl.vapi b/vapi/gsl.vapi\nin...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/vapi/gsl.vapi b/vapi/gsl.vapi\n@@...,Write a concise Git commit message (imperative...


In [8]:
train.reset_index(inplace=True)
val.reset_index(inplace=True)
test.reset_index(inplace=True)

# Create Datasets and DataLoaders
train_dataset = preprocesser.SentencePairDataset(train, bert_tokenizer, codebert_tokenizer,message='msgs',command='diff_compact',label='labels')
val_dataset = preprocesser.SentencePairDataset(val, bert_tokenizer, codebert_tokenizer,message='msgs',command='diff_compact',label='labels')
test_dataset = preprocesser.SentencePairDataset(test, bert_tokenizer, codebert_tokenizer,message='msgs',command='diff_compact',label='labels')
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)



# # Initialize the model
# model = em.CombinedModel(bert_model, codebert_model, bert_tokenizer, codebert_tokenizer)

# # Train the model
# model.trainer(train_loader, val_loader,num_epochs=10)
# for batch in train_loader:
#     print(batch)


base_model1 = cm.BaseModel(bert_model)
base_model2 = cm.BaseModel(codebert_model)

# Create stacking model
combined_model = cm.CombinedModel(base_model1, base_model2)
# Train the model
combined_model.trainer(train_loader, val_loader,num_epochs=6, patience=3)

Epoch 1/6:   3%|▎         | 1/39 [00:01<00:56,  1.48s/batch]

Epoch 0, average loss: 1.16665917634964


evaluate: 100%|██████████| 9/9 [00:04<00:00,  2.13batch/s]

Epoch 0, validate accuracy score: 0.6666666666666666



Epoch 2/6:   3%|▎         | 1/39 [00:01<00:39,  1.04s/batch]

...

Epoch 4, validate accuracy score: 0.8202247191011236



Epoch 6/6:   3%|▎         | 1/39 [00:01<00:39,  1.05s/batch]

Epoch 5, average loss: 0.03583128098398447


evaluate: 100%|██████████| 9/9 [00:03<00:00,  2.31batch/s]

Epoch 5, validate accuracy score: 0.7865168539325843


In [9]:
test_dataset = preprocesser.SentencePairDataset(test, bert_tokenizer, codebert_tokenizer,message='msgs',command='diff_compact',label='labels')
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [10]:
test_acc, test_labels, test_probabilities, test_embeddings, test_predictions = combined_model.evaluate(test_loader)

evaluate: 100%|██████████| 9/9 [00:04<00:00,  2.12batch/s]

Validation Accuracy: 0.7947761194029851
Precision: 0.7965195666688204
Recall: 0.7947761194029851
F1-Score: 0.7940282493389623


In [11]:
from sklearn import metrics

In [12]:
cm = metrics.confusion_matrix(test_labels, test_predictions)
cm

array([[58,  6, 15],
       [ 6, 77,  6],
       [ 6, 16, 78]])

In [13]:
import torch
# 保存
torch.save(
    {
        "model_state_dict": combined_model.state_dict(),
        "num_labels": 3,
    },
    "./reward_model_combined.pt",
)

# 加载
ckpt = torch.load("reward_model_combined.pt")
combined_model.load_state_dict(ckpt["model_state_dict"])

<All keys matched successfully>

In [14]:
# 加载
# reward_model = torch.load("reward_model_combined.pth")
# combined_model.load_state_dict(ckpt["model_state_dict"])

In [15]:
#发送多种类型的邮件
from email.mime.multipart import MIMEMultipart
import smtplib

from email.mime.text import MIMEText
msg_from = '915803745@qq.com'  # 发送方邮箱
passwd = 'vcuosuurrgkfbdai'   #就是上面的授权码
 
# to= ['g.zhang@gotion.com', 'j.tong@gotion.com'] #接受方邮箱
to= ['j.tong@gotion.com'] #接受方邮箱
#设置邮件内容
#MIMEMultipart类可以放任何内容
msg = MIMEMultipart()
conntent=f"强化学习"
#把内容加进去
msg.attach(MIMEText(conntent,'plain','utf-8'))
 
#设置邮件主题
msg['Subject']="强化学习模型训练完毕"
 
#发送方信息
msg['From']=msg_from
 
#开始发送
 
#通过SSL方式发送，服务器地址和端口
s = smtplib.SMTP_SSL("smtp.qq.com", 465)
# 登录邮箱
s.login(msg_from, passwd)
#开始发送
s.sendmail(msg_from,to,msg.as_string())
print("强化学习模型训练完毕")

强化学习模型训练完毕
